In [7]:
import os
from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain_openai import ChatOpenAI

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai  import ChatGoogleGenerativeAI
from langchain.agents.agent_types import AgentType
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory

import gradio as gr
#search for available flsht to roma on 20/06/2025
load_dotenv()

langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
open_api_key=os.getenv("OPEN_API_KEY")



In [21]:

# search_flights
@tool
def search_flights(location: str, date: str) -> str:
    """Search for flights based on location and date."""
    return f"Flightsaaaaa available from {location} on {date}: Flight AZjihad-101 at 10:00 AM, Flight AZ-202 at 5:00 PM."
@tool
#book a flight
def book_flight(flight_id: str, passenger_name: str) -> str:
    """Book a flight with a given flight ID and passenger name."""
    return f"Flight {flight_id} successfully booked for {passenger_name}."
@tool
def handle_complaint(complaint_text: str) -> str:
    """
    Records the user's complaint into a file.
    do not invent or give reasons , just provide the input user and fix grammer only when needed
    Assumes the LLM has already extracted the pure complaint text.
    """
    with open("customer_complaint.txt", "a") as file:  # 'a' = append mode
        file.write(complaint_text + "\n---\n")  # Add separator between complaints
    return "Your complaint has been recorded. We appreciate your feedback."

In [ ]:
# here i used the agent to automaticlly mange the flows between 2 tools
#scnario 1 : Book a flight to jihad for roma 
#scnario 2: the food on the flight is so bad

#The LLM reads my input and understands the logical dependencies between search_flights,book_flight
#Calls search_flights() then Chooses the best flight now hardcoded

#pass the arguments returned fro search_flights to  book_flight() using Aggnet (10x for Agents)
# returned messagge




tools = [search_flights, book_flight, handle_complaint]



 #just telling the model what to do a semantic guidance dn not as setup
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a helpful travel assistant. Use tools when needed. "
     "If the user is making a complaint, use the 'handle_complaint' tool. "
     "For flight search or booking, use the appropriate tools."),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

openai_model_id = "gpt-4o-mini"
##llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
llm = ChatOpenAI(
    model=openai_model_id,         
    temperature=0,# make sure the same reply and standard responses especially in foundation
    api_key=open_api_key
)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    memory = memory,
    verbose=True
)

# Chat function for Gradio
def chat_with_agent(message, history):
    """Filter out messages requesting travel to Kayan"""
    if "kayan" in message.lower():
        return " Sorry, booking to 'Kayan' is not allowed."

    # Normal agent flow
    return agent_executor.invoke({"input": message})["output"]

if __name__ == "__main__":
    import gradio as gr
    iface = gr.ChatInterface(
        fn = chat_with_agent,
        title="Flight  search",
        description="Chat with an AI find best flight!",
    )
    iface.launch()


d:\desktop\TFAjava\AI\TheAIEngineers-Exercises\testenv1\Lib\site-packages\gradio\chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...

Invoking: `handle_complaint` with `{'complaint_text': 'The drink is almost good and it can be better.'}`


Your complaint has been recorded. We appreciate your feedback.Your complaint has been recorded. We appreciate your feedback.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `handle_complaint` with `{'complaint_text': 'also the drink'}`


Your complaint has been recorded. We appreciate your feedback.Your complaint has been recorded. Thank you for your feedback.

> Finished chain.
